# 🌍 Multilingual LSTM Translator

**Name:** Jivesh Gupta  
**Roll No:** 33  

---

## 📌 Project Description

This project implements a **Multilingual Neural Machine Translation (NMT)** system using a **Sequence-to-Sequence (Seq2Seq) architecture with LSTM networks**.

The model supports translation between:

- English ↔ Hindi  
- English ↔ Punjabi  
- Hindi ↔ English  
- Punjabi ↔ English

In [1]:
!pip install tensorflow pandas numpy scikit-learn

import pandas as pd
import numpy as np
import tensorflow as tf
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [13]:
from google.colab import files
uploaded = files.upload()

df = pd.read_csv("/content/trilingual_dataset_20000_high_diversity.csv")
df.head()

,english,hindi,punjabi
0,hello,नमस्ते,ਸਤ ਸ੍ਰੀ ਅਕਾਲ
1,thanks,धन्यवाद,ਧੰਨਵਾਦ
2,sorry,माफ़ कीजिए,ਮਾਫ਼ ਕਰਨਾ
3,yes,हाँ,ਹਾਂ
4,no,नहीं,ਨਹੀਂ


In [14]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"\s+", " ", text)
    return text.strip()

df["english"] = df["english"].apply(clean_text)
df["hindi"] = df["hindi"].apply(clean_text)
df["punjabi"] = df["punjabi"].apply(clean_text)

In [38]:
pairs = []

for _, row in df.iterrows():

    # EN → HI
    pairs.append(("<2hi> " + row["english"],
                  "<sos> " + row["hindi"] + " <eos>"))

    # EN → PA
    pairs.append(("<2pa> " + row["english"],
                  "<sos> " + row["punjabi"] + " <eos>"))

    # HI → EN
    pairs.append(("<2en> " + row["hindi"],
                  "<sos> " + row["english"] + " <eos>"))

    # PA → EN
    pairs.append(("<2en> " + row["punjabi"],
                  "<sos> " + row["english"] + " <eos>"))

input_texts = [p[0] for p in pairs]
target_texts = [p[1] for p in pairs]

print("Total training pairs:", len(input_texts))

Total training pairs: 20216


In [41]:
tokenizer = Tokenizer(
    num_words=8000,
    oov_token="<OOV>",
    filters=''   # <-- THIS IS THE FIX
)
tokenizer.fit_on_texts(input_texts + target_texts)

In [43]:
input_sequences = tokenizer.texts_to_sequences(input_texts)
target_sequences = tokenizer.texts_to_sequences(target_texts)

In [44]:
max_input_len = max(len(seq) for seq in input_sequences)
max_target_len = max(len(seq) for seq in target_sequences)

encoder_input_data = pad_sequences(input_sequences, maxlen=max_input_len, padding="post")
decoder_input_data = pad_sequences(target_sequences, maxlen=max_target_len, padding="post")

decoder_target_data = np.zeros_like(decoder_input_data)
decoder_target_data[:, :-1] = decoder_input_data[:, 1:]

In [45]:
print("<sos>" in tokenizer.word_index)
print("<eos>" in tokenizer.word_index)

True
True


In [46]:
vocab_size = 8000
tokenizer = Tokenizer(
    num_words=vocab_size,
    oov_token="<OOV>",
    filters=''
)
tokenizer.fit_on_texts(input_texts + target_texts)

input_sequences = tokenizer.texts_to_sequences(input_texts)
target_sequences = tokenizer.texts_to_sequences(target_texts)

In [47]:
max_input_len = max(len(seq) for seq in input_sequences)
max_target_len = max(len(seq) for seq in target_sequences)

encoder_input_data = pad_sequences(input_sequences, maxlen=max_input_len, padding="post")
decoder_input_data = pad_sequences(target_sequences, maxlen=max_target_len, padding="post")

decoder_target_data = np.zeros_like(decoder_input_data)

decoder_target_data[:, :-1] = decoder_input_data[:, 1:]

In [48]:
X_train_enc, X_val_enc, X_train_dec, X_val_dec, y_train, y_val = train_test_split(
    encoder_input_data,
    decoder_input_data,
    decoder_target_data,
    test_size=0.2,
    random_state=42
)

In [52]:
embedding_dim = 128
latent_dim = 256

# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(vocab_size, embedding_dim, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True, dropout=0.4, recurrent_dropout=0)
_, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(vocab_size, embedding_dim, mask_zero=True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,
                    dropout=0.4, recurrent_dropout=0)

decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)

decoder_dense = Dense(vocab_size, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
import tensorflow as tf

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name="accuracy")]
)
model.summary()

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_14      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_15      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_10        │ (None, None, 128) │  1,024,000 │ input_layer_14[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_12        │ (None, None)      │          0 │ input_layer_14[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_11        │ (None, None, 128) │  1,024,000 │ input_layer_15[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_10 (LSTM)      │ [(None, 256),     │    394,240 │ embedding_10[0][… │
│                     │ (None, 256),      │            │ not_equal_12[0][… │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_11 (LSTM)      │ [(None, None,     │    394,240 │ embedding_11[0][… │
│                     │ 256), (None,      │            │ lstm_10[0][1],    │
│                     │ 256), (None,      │            │ lstm_10[0][2]     │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, None,      │  2,056,000 │ lstm_11[0][0]     │
│                     │ 8000)             │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,892,480 (18.66 MB)

 Trainable params: 4,892,480 (18.66 MB)

 Non-trainable params: 0 (0.00 B)

In [53]:
callbacks = [
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3)
]

In [54]:
history = model.fit(
    [X_train_enc, X_train_dec],
    np.expand_dims(y_train, -1),
    validation_data=([X_val_enc, X_val_dec], np.expand_dims(y_val, -1)),
    batch_size=64,
    epochs=20,
    callbacks=callbacks
)

Epoch 1/20
253/253 ━━━━━━━━━━━━━━━━━━━━ 15s 47ms/step - accuracy: 0.3044 - loss: 4.7961 - val_accuracy: 0.2805 - val_loss: 2.2842 - learning_rate: 0.0010
Epoch 2/20
253/253 ━━━━━━━━━━━━━━━━━━━━ 11s 45ms/step - accuracy: 0.3151 - loss: 1.8623 - val_accuracy: 0.4011 - val_loss: 1.1059 - learning_rate: 0.0010
Epoch 3/20
253/253 ━━━━━━━━━━━━━━━━━━━━ 11s 45ms/step - accuracy: 0.4109 - loss: 1.0390 - val_accuracy: 0.4237 - val_loss: 0.8956 - learning_rate: 0.0010
Epoch 4/20
253/253 ━━━━━━━━━━━━━━━━━━━━ 11s 44ms/step - accuracy: 0.4297 - loss: 0.8601 - val_accuracy: 0.4493 - val_loss: 0.7637 - learning_rate: 0.0010
Epoch 5/20
253/253 ━━━━━━━━━━━━━━━━━━━━ 11s 44ms/step - accuracy: 0.4579 - loss: 0.7327 - val_accuracy: 0.4941 - val_loss: 0.6302 - learning_rate: 0.0010
Epoch 6/20
253/253 ━━━━━━━━━━━━━━━━━━━━ 11s 45ms/step - accuracy: 0.4999 - loss: 0.5953 - val_accuracy: 0.5394 - val_loss: 0.4538 - learning_rate: 0.0010
Epoch 7/20
253/253 ━━━━━━━━━━━━━━━━━━━━ 11s 45ms/step - accuracy: 0.5550 - l

In [55]:
model.save("multilingual_lstm_translator.h5")

import pickle
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

In [56]:
len(tokenizer.word_index)

232

In [57]:
encoder_model = Model(encoder_inputs, encoder_states)

In [58]:
latent_dim = 256  # same as training

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = dec_emb_layer(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(
    dec_emb2, initial_state=decoder_states_inputs
)

decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2
)

In [65]:
index_word = tokenizer.index_word
word_index = tokenizer.word_index

def translate_text(input_text, source_lang, target_lang):

    if source_lang == target_lang:
        return "Source and Target language cannot be same."

    # Direction token mapping
    direction_tokens = {
        ("English", "Hindi"): "<2hi>",
        ("English", "Punjabi"): "<2pa>",
        ("Hindi", "English"): "<2en>",
        ("Punjabi", "English"): "<2en>",
        ("Hindi", "Punjabi"): "<2pa>",
        ("Punjabi", "Hindi"): "<2hi>",
    }

    token = direction_tokens.get((source_lang, target_lang))

    if token is None:
        return "Translation direction not supported."

    # Add direction token
    input_text = token + " " + input_text.strip()

    # Tokenize
    sequence = tokenizer.texts_to_sequences([input_text])
    sequence = pad_sequences(sequence, maxlen=max_input_len, padding="post")

    # Encode
    states_value = encoder_model.predict(sequence)

    # Prepare decoder input
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tokenizer.word_index["<sos>"]

    stop_condition = False
    decoded_sentence = ""

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = tokenizer.index_word.get(sampled_token_index, "")

        if sampled_word == "<eos>" or len(decoded_sentence.split()) > 20:
            stop_condition = True
        else:
            decoded_sentence += sampled_word + " "

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        states_value = [h, c]

    return decoded_sentence.strip()

In [66]:
model.summary()

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_14      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_15      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_10        │ (None, None, 128) │  1,024,000 │ input_layer_14[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_12        │ (None, None)      │          0 │ input_layer_14[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_11        │ (None, None, 128) │  1,024,000 │ input_layer_15[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_10 (LSTM)      │ [(None, 256),     │    394,240 │ embedding_10[0][… │
│                     │ (None, 256),      │            │ not_equal_12[0][… │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_11 (LSTM)      │ [(None, None,     │    394,240 │ embedding_11[0][… │
│                     │ 256), (None,      │            │ lstm_10[0][1],    │
│                     │ 256), (None,      │            │ lstm_10[0][2]     │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, None,      │  2,056,000 │ lstm_11[0][0]     │
│                     │ 8000)             │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 14,677,442 (55.99 MB)

 Trainable params: 4,892,480 (18.66 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 9,784,962 (37.33 MB)

In [67]:
print(latent_dim)

256


In [68]:
print("<sos>" in tokenizer.word_index)
print("<eos>" in tokenizer.word_index)

True
True


In [71]:
model.save("model.h5")

import pickle
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

In [69]:
!pip install gradio

In [70]:
import gradio as gr

with gr.Blocks(theme=gr.themes.Soft()) as demo:

    gr.Markdown(
        """
        # 🌍 Multilingual LSTM Translator
        Translate between English, Hindi, and Punjabi using LSTM Seq2Seq Model
        """
    )

    with gr.Row():
        with gr.Column():
            input_text = gr.Textbox(
                label="Enter Text",
                placeholder="Type your sentence here...",
                lines=4
            )

            source_lang = gr.Dropdown(
                ["English", "Hindi", "Punjabi"],
                label="Source Language",
                value="English"
            )

            target_lang = gr.Dropdown(
                ["English", "Hindi", "Punjabi"],
                label="Target Language",
                value="Hindi"
            )

            submit_btn = gr.Button("Translate", variant="primary")

        with gr.Column():
            output_text = gr.Textbox(
                label="Translated Output",
                lines=4
            )

    submit_btn.click(
        translate_text,
        inputs=[input_text, source_lang, target_lang],
        outputs=output_text
    )

demo.launch()

/tmp/ipython-input-2488/2495267343.py:3: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fba5e38261d55e497f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
